In [2]:
import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras import *
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import *
import random
#import ../utils.py
import sys
sys.path.append('../')
from utils import *

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

def reduce(dataset):
    return dataset[:int(dataset.shape[0]/2)]

x_train = reduce(x_train)
y_train = reduce(y_train)

Y_train = y_train
Y_test = y_test


num_classes = 10 
x_train = x_train/255.
y_train = utils.to_categorical(y_train, num_classes) 

x_test = x_test/255.
y_test = utils.to_categorical(y_test, num_classes) 

In [4]:
def build_model(weight_decay= 0.0005, x_shape=[32,32,3], num_classes=10):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

    model = Sequential()
#     model.add(Input(shape=()))

    model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(4096, activation= 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation= 'relu')) 
    model.add(Dropout(0.5))

    return model

In [5]:
########################################## Building siamese network ###################################


def build_siamese_vgg_model(shape, class_num):
    inputs1 = Input(shape)
    inputs2 = Input(shape)
    

    x1 = build_model()
    x2 = build_model()

    x = layers.concatenate([x1.outputs[0], x2.outputs[0]])
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)  

    # build the model
    model = Model([x1.inputs[0], x2.inputs[0]], outputs)
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

model = build_siamese_vgg_model(x_train[0].shape, num_classes)



/home/x/miniconda3/envs/ml4/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-13 19:05:32.886938: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8596 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
def make_pairs(images, labels):
    pairImages = []
    pairLabels = []

    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]

    for idxA in range(len(images)):
        currentImage = images[idxA]
        label = labels[idxA]
#         print(currentImage.shape, label)

        
        for idxB in random.sample(list(idx[label[0]]), 1):
            posImage = images[idxB]
#             print(posImage.shape)
#             print('\n\n\n')


            pairImages.append(np.array([currentImage, posImage]))
            pairLabels.append(np.array(label))
        
    return np.array(pairImages), np.array(pairLabels)

In [7]:
######################################################################################################

x_train_siamese, y_train_siamese = make_pairs(x_train, Y_train)
x_test_siamese, y_test_siamese = make_pairs(x_test, Y_test)

y_train_siamese = utils.to_categorical(y_train_siamese, num_classes) 
y_test_siamese = utils.to_categorical(y_test_siamese, num_classes) 

x_train_both = [x_train_siamese[:,0,:], x_train_siamese[:,1,:]]
x_test_both = [x_test_siamese[:,0,:], x_test_siamese[:,1,:]]


print('Images')
print(np.array(x_train_both).shape)
print(np.array(x_test_both).shape)


print('Labels')
print(y_train_siamese.shape)
print(y_test_siamese.shape)

Images
(2, 25000, 32, 32, 3)
(2, 10000, 32, 32, 3)
Labels
(25000, 10)
(10000, 10)


In [8]:


# print(model.summary())

# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)

history3 = model.fit(x_train_both, y_train_siamese, epochs=400, batch_size=512, shuffle=True,
         validation_data=(x_test_both, y_test_siamese),callbacks=[])


print('Saving model')


Epoch 1/400
49/49 [==============================] - 8s 169ms/step - loss: 1.0121 - accuracy: 0.9420 - val_loss: 1.5517 - val_accuracy: 0.7930
Epoch 2/400
49/49 [==============================] - 8s 161ms/step - loss: 1.0477 - accuracy: 0.9344 - val_loss: 1.6413 - val_accuracy: 0.8016
Epoch 3/400
49/49 [==============================] - 8s 161ms/step - loss: 1.0357 - accuracy: 0.9399 - val_loss: 1.4607 - val_accuracy: 0.8217
Epoch 4/400
49/49 [==============================] - 8s 160ms/step - loss: 1.0163 - accuracy: 0.9442 - val_loss: 1.3717 - val_accuracy: 0.8471
Epoch 5/400
49/49 [==============================] - 8s 160ms/step - loss: 1.0051 - accuracy: 0.9456 - val_loss: 1.4613 - val_accuracy: 0.8275
Epoch 6/400
49/49 [==============================] - 8s 160ms/step - loss: 1.0276 - accuracy: 0.9403 - val_loss: 1.3165 - val_accuracy: 0.8489
Epoch 7/400
49/49 [==============================] - 8s 161ms/step - loss: 1.0449 - accuracy: 0.9381 - val_loss: 1.4011 - val_accuracy: 0.8356

In [9]:
model.save('cifar_vgg/models/Cifar10-vgg-siamese-small-400ep.h5')


In [8]:
model = tf.keras.models.load_model("cifar_vgg/models/Cifar10-vgg-siamese-small-400ep.h5")

results = full_evaluate(model, x_test_both, Y_test) # id15

print()



/home/x/miniconda3/envs/ml4/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
I0000 00:00:1718298350.309856  388609 service.cc:145] XLA service 0x79627c0264b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718298350.309883  388609 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-06-13 19:05:50.333188: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-13 19:05:50.472607: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1718298350.893917  388667 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_890', 192 bytes spill stores, 192 bytes spill loads

I0000 00:00:

 20/313 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step

I0000 00:00:1718298351.767860  388609 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


311/313 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

I0000 00:00:1718298355.340552  388719 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_890', 128 bytes spill stores, 128 bytes spill loads



313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step
accuracy_score: 0.8303
f1_score_macro: 0.8334206905935595
f1_score_weighted: 0.8271793094064406



In [9]:
from tensorflow.keras.utils import plot_model
model = tf.keras.models.load_model("cifar_vgg/models/Cifar10-vgg-siamese-small-400ep.h5")
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

/home/x/miniconda3/envs/ml4/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/x/miniconda3/envs/ml4/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
